In [7]:
from sklearn.datasets import load_diabetes
from sklearn.linear_model import Lasso, ElasticNet
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse
from sklearn.dummy import DummyRegressor
from sklearn.preprocessing import StandardScaler
from pathlib import Path
import yaml
from copy import deepcopy
from sklearn.metrics import mean_squared_error
from JOPLEn.enums import NormType, CellModel
from JOPLEn.singletask import l21_norm, linf1_norm
from JOPLEn.singletask import JOPLEn

import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

from ax.service.ax_client import AxClient, ObjectiveProperties

import warnings
import logging
from ax.utils.common.logger import ROOT_STREAM_HANDLER

ROOT_STREAM_HANDLER.setLevel(logging.ERROR)
from collections import defaultdict


# Hide future warnings because ax uses deprecated functions from pandas
warnings.simplefilter(action="ignore", category=FutureWarning)
# Hide unfixable warning from ax (warns about default behavior but there isn't
# a clear way to turn the warning off)
warnings.simplefilter(action="ignore", category=UserWarning)


LST_DATASETS = ["boston", "diabetes", "riboflavin"]

CACHE_DIR = Path("ax_runs") / "st_selection"
CACHE_DIR.mkdir(parents=True, exist_ok=True)
DS_PATH = Path("..") / "datasets"

PARAM_PATH = Path(".") / "parameters"
PLOT_PATH = Path(".") / "plots"


In [6]:
model_info = {}

for file in PARAM_PATH.glob("*.yaml"):
    model_info[file.stem] = yaml.safe_load(open(file, "r"))


In [ ]:
def rmse(y_true, y_pred):
    return mean_squared_error(y_true, y_pred, squared=False)


def train_sklearn(
    ModelClass,
    params,
    x_train,
    y_train,
    x_val,
    y_val,
    x_test,
    y_test,
):
    model = ModelClass(**params)
    model.fit(x_train, y_train.flatten())

    return {
        "train_error": float(rmse(y_train.flatten(), model.predict(x_train))),
        "val_error": float(rmse(y_val.flatten(), model.predict(x_val))),
        "test_error": float(rmse(y_test.flatten(), model.predict(x_test))),
        "model": model,
    }


def train_lasso(params, x_train, y_train, x_val, y_val, x_test, y_test):
    res = train_sklearn(
        Lasso,
        params,
        x_train,
        y_train,
        x_val,
        y_val,
        x_test,
        y_test,
    )

    eps = 1e-6

    return {
        "density": np.mean(np.abs(res["model"].coef_) > eps),
        **res,
    }


def dummy_regressor(x_train, y_train, x_val, y_val, x_test, y_test):
    dummy = DummyRegressor(strategy="mean")
    dummy.fit(x_train, y_train.flatten())

    return {
        "train_error": float(rmse(y_train.flatten(), dummy.predict(x_train))),
        "val_error": float(rmse(y_val.flatten(), dummy.predict(x_val))),
        "test_error": float(rmse(y_test.flatten(), dummy.predict(x_test))),
    }


def train_joplen(
    ModelType,
    params,
    x_train,
    y_train,
    x_val,
    y_val,
    x_test,
    y_test,
):
    params = deepcopy(params)

    initial_params = {
        "partitioner": eval(params.pop("partitioner")),
        "n_cells": params.pop("n_cells"),
        "n_partitions": params.pop("n_partitions"),
        "random_state": params.pop("random_state"),
    }

    initial_params["cell_model"] = eval(params.pop("cell_model"))
    assert initial_params["cell_model"] == CellModel.linear

    model = ModelType(**initial_params)

    assert "norm_type" in params

    params["norm_type"] = eval(params["norm_type"])

    history = model.fit(
        x_train,
        y_train,
        val_x=x_val,
        val_y=y_val,
        **params,
    )

    if params["norm_type"] == NormType.L21:
        norm = l21_norm
    elif params["norm_type"] == NormType.LINF1:
        norm = linf1_norm
    else:
        raise NotImplementedError()

    return {
        "train_error": float(rmse(y_train, model.predict(x_train))),
        "val_error": float(rmse(y_val, model.predict(x_val))),
        "test_error": float(rmse(y_test, model.predict(x_test))),
        "model": model,
        "epochs": len(history),
        "density": float((norm(model.w.get()) > 1e-6).mean()),
    }


In [ ]:
def pareto_front(
    score: np.ndarray, density: np.ndarray
) -> tuple[np.ndarray, np.ndarray]:
    unique_density = np.unique(density)

    pareto_score = []
    pareto_density = []

    for d in unique_density:
        mask = density == d
        pareto_score.append(np.min(score[mask]))
        pareto_density.append(d)

    pareto_score = np.array(pareto_score)
    pareto_density = np.array(pareto_density)

    args = np.argsort(pareto_density)

    return pareto_density[args], pareto_score[args]


def score(dummy_res, model_res, alpha):
    res = {
        "error": {
            "train": model_res["train_error"] / dummy_res["train_error"],
            "val": model_res["val_error"] / dummy_res["val_error"],
            "test": model_res["test_error"] / dummy_res["test_error"],
        },
        "density": model_res["density"],
    }

    res["pareto"] = {
        "train": alpha * res["density"] + (1 - alpha) * res["error"]["train"],
        "val": alpha * res["density"] + (1 - alpha) * res["error"]["val"],
        "test": alpha * res["density"] + (1 - alpha) * res["error"]["test"],
    }

    return res


In [ ]:
train_fn = {
    Lasso.__name__: train_lasso,
    JOPLEn.__name__: train_joplen,
}


def optimize_model(model_info, ds_path, n_trials, minimize, loss_type, alpha):
    ds_name = ds_path.name
    params = model_info["parameters"]

    dir_path = CACHE_DIR / model_info["dir_name"] / ds_name
    exp_path = dir_path / "experiment.json"
    metadata_path = dir_path / "metadata.yaml"

    if metadata_path.exists():
        with open(metadata_path, "r") as f:
            metadata = yaml.load(f, Loader=yaml.FullLoader)
        return metadata

    x_train = np.loadtxt(ds_path / "x_train.csv", delimiter=",")
    x_val = np.loadtxt(ds_path / "x_val.csv", delimiter=",")
    x_test = np.loadtxt(ds_path / "x_test.csv", delimiter=",")
    y_train = np.loadtxt(ds_path / "y_train.csv", delimiter=",")
    y_val = np.loadtxt(ds_path / "y_val.csv", delimiter=",")
    y_test = np.loadtxt(ds_path / "y_test.csv", delimiter=",")

    dummy_res = dummy_regressor(
        x_train,
        y_train,
        x_val,
        y_val,
        x_test,
        y_test,
    )

    if not exp_path.exists():
        ax_client = AxClient(
            random_seed=0,
            verbose_logging=False,
        )

        ax_client.create_experiment(
            name=model_info["model"],
            parameters=params,
            objectives={loss_type: ObjectiveProperties(minimize=minimize)},
            overwrite_existing_experiment=True,
        )

        for _ in range(n_trials):
            round_params, trial_index = ax_client.get_next_trial()

            try:
                model_res = train_fn[model_info["model"]](
                    round_params,
                    x_train=x_train,
                    y_train=y_train,
                    x_val=x_val,
                    y_val=y_val,
                    x_test=x_test,
                    y_test=y_test,
                )
                model_score = score(dummy_res, model_res, alpha)
                ax_client.complete_trial(
                    trial_index=trial_index, raw_data=model_score["pareto"]["val"]
                )
            except ValueError as e:
                print(e)
                ax_client.abandon_trial(
                    trial_index=trial_index,
                    reason=str(e),
                )
    else:
        ax_client = AxClient.load_from_json_file(exp_path)

    best_parameters, _ = ax_client.get_best_parameters()

    model_res = train_fn[model_info["model"]](
        best_parameters,
        x_train=x_train,
        y_train=y_train,
        x_val=x_val,
        y_val=y_val,
        x_test=x_test,
        y_test=y_test,
    )
    model_score = score(dummy_res, model_res, alpha)

    metadata = {
        "model": {
            "name": model_info["model"],
            "results": model_res,
            "score": model_score,
        },
        "dummy": {
            "name": "DummyRegressor",
            **dummy_res,
        },
    }

    return metadata


In [ ]:
reg_res = defaultdict(dict)

alphas = np.linspace(0, 1, 6, endpoint=True)
datasets = [DS_PATH / ds for ds in LST_DATASETS]
itr = tqdm(alphas)

for alpha in itr:
    for ds_path in datasets:
        for file_name, info in model_info.items():
            if "st_selection" not in info["experiments"]:
                continue

            model_str = f"{file_name} on {ds_path.name}"
            itr.set_description(f"Running {model_str : <50}")
            res = optimize_model(info, ds_path, 10, True, "rmse", alpha)

            if res is not None:
                reg_res[info["name"]][ds_path.name].append(res)

reg_res = dict(reg_res)
